In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt

In [2]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price, amm-price, jump-multiplier

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e' or base == 'Delta':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b]),
 's': dict_values([s_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s])}

In [3]:
simp = True;

## Target function

we want to minimize the effective price, given a set of exponents for the buying
and selling asset each.

In [4]:
# linear
def deltaBySpot_(s,v, b, w, j, e, Delta, a):
  f = (s - (v + b) * w) / w
  if simp:
    f = simplify(f)
  return f

deltaBySpot = {asset: deltaBySpot_(*all_symbols[asset]) for asset in assets}
deltaBySpot['b']

-b_b + s_b/w_b - v_b

In [5]:
# convex
def spotByExp_(s, v, b, w, j, e, Delta, a):
  f = a * ((1 + 1/j) ** e)
  if simp:
    f = simplify(f)
  return f
spotByExp = {asset: spotByExp_(*all_symbols[asset]) for asset in assets}
spotByExp['b']

a_b*((j_b + 1)/j_b)**e_b

In [6]:
# convex
def deltaByExp_(asset, s, v, b, w, j, e, Delta, a):
    f = deltaBySpot[asset].subs(s, spotByExp[asset])
    if simp:
        f = simplify(f)
    return f

deltaByExp = {asset: deltaByExp_(asset, *all_symbols[asset]) for asset in assets}
deltaByExp['b']

a_b*j_b**(-e_b)*(j_b + 1)**e_b/w_b - b_b - v_b

In [7]:
#convex;
eff = Delta_s / -Delta_b;
eff;

-Delta_s/Delta_b

In [8]:
Delta_b_by_Exp_s = -deltaByExp['s'] * spotByExp['b'] / spotByExp['s']
if simp:
    Delta_b_by_Exp_s = simplify(Delta_b_by_Exp_s)
Delta_b_by_Exp_s

a_b*((j_b + 1)/j_b)**e_b*(j_s + 1)**(-e_s)*(-a_s*(j_s + 1)**e_s + j_s**e_s*w_s*(b_s + v_s))/(a_s*w_s)

In [9]:
Delta_s_by_Exp_b = -deltaByExp['b'] * spotByExp['s'] / spotByExp['b']
if simp:
    Delta_s_by_Exp_b = simplify(Delta_s_by_Exp_b)
Delta_s_by_Exp_b

a_s*((j_s + 1)/j_s)**e_s*(j_b + 1)**(-e_b)*(-a_b*(j_b + 1)**e_b + j_b**e_b*w_b*(b_b + v_b))/(a_b*w_b)

In [10]:
effByExp_b = eff.subs({Delta_b: deltaByExp['b'], Delta_s: Delta_s_by_Exp_b})
if simp:
    effByExp_b = simplify(effByExp_b)
effByExp_b

a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s/a_b

In [11]:
effByExp_s = eff.subs({Delta_b: Delta_b_by_Exp_s, Delta_s: deltaByExp['s']})
if simp:
    effByExp_s = simplify(effByExp_s)
effByExp_s

a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s/a_b

In [12]:
simplify(effByExp_b - effByExp_s);

0

In [13]:
effByExps = effByExp_b;

In [14]:
H = hessian(effByExps, [e_b, e_s]);
H;

Matrix([
[                a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log(j_b/(j_b + 1))**2/a_b, a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log(j_b/(j_b + 1))*log((j_s + 1)/j_s)/a_b],
[a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log(j_b/(j_b + 1))*log((j_s + 1)/j_s)/a_b,                 a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log((j_s + 1)/j_s)**2/a_b]])

In [15]:
topLeft = simplify(H[0, 0]);
topLeft;

a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log(j_b/(j_b + 1))**2/a_b

-> positive

In [16]:
det = H.det();
det;

0

## inequality-constraints

- the exponents must adhere to their upper (buying) resp. lower (selling) bounds
  given by our equation
- need to buy and sell minimum amounts (TODO)
- cannot buy more than the available balance
- cannot sell more than maxSelling
- the spot prices must not exceed maxInteger
- bonus: the total number of multiplications for both exponentiations must not
  exceed expLimit (TODO)

In [19]:
def expBound_(asset, s, v, b, w, j, e, Delta, a):
    e_bound = log(w * (v + b) /a, 1 + 1/j) # constant wrt e
    if simp:
        e_bound = simplify(e_bound)
    if asset == 'b':
        f = e - e_bound # <=! 0 (upper bound)
    else:
        f = e_bound - e # <=! 0 (lower bound)
    if simp:
        f = simplify(f)
    return f

expBound = {asset: expBound_(asset, *all_symbols[asset]) for asset in assets}
expBound['b']

e_b - log(a_b**(-1/log((j_b + 1)/j_b))*(w_b*(b_b + v_b))**(1/log((j_b + 1)/j_b)))

In [20]:
expBound["s"];

-e_s + log(a_s**(-1/log((j_s + 1)/j_s))*(w_s*(b_s + v_s))**(1/log((j_s + 1)/j_s)))

In [21]:
maxBuyingBound = -Delta_b - b_b # <=! 0
maxBuyingBound = maxBuyingBound.subs({Delta_b: deltaByExp['b']})
if simp:
    maxBuyingBound = simplify(maxBuyingBound)
maxBuyingBound

-a_b*j_b**(-e_b)*(j_b + 1)**e_b/w_b + v_b

In [22]:
max_s, max_I = symbols('max_s, max_I', positive=True, integer=True)

maxSellingBound = Delta_s - max_s # <=! 0
maxSellingBound = maxSellingBound.subs({Delta_s: deltaByExp['s']})
if simp:
    maxSellingBound = simplify(maxSellingBound)
maxSellingBound

a_s*j_s**(-e_s)*(j_s + 1)**e_s/w_s - b_s - max_s - v_s

In [23]:
def maxSpotBound_(asset, s, v, b, w, j, e, Delta, a):
  return spotByExp[asset] - max_I # <=! 0

maxSpotBound = {asset: maxSpotBound_(asset, *all_symbols[asset]) for asset in assets}
maxSpotBound['b']

a_b*((j_b + 1)/j_b)**e_b - max_I

## Lagrangian

In [24]:
l1, l2, l3, l4, l5, l6 = symbols('lambda_1 lambda_2 lambda_3 lambda_4 lambda_5 lambda_6', positive=True)

# l3+ have no effect
L = effByExps + l1 * expBound['b'] + l2 * expBound['s'] #+ l3 * maxBuyingBound + l4 * maxSellingBound + l5 * maxSpotBound['b'] + l6 * maxSpotBound['s']
if simp:
    L = simplify(L)
L

e_b*lambda_1 - e_s*lambda_2 - lambda_1*log(a_b**(-1/log(1 + 1/j_b))*(b_b*w_b + v_b*w_b)**(1/log(1 + 1/j_b))) + lambda_2*log(a_s**(-1/log(1 + 1/j_s))*(b_s*w_s + v_s*w_s)**(1/log(1 + 1/j_s))) + a_s*j_b**e_b*j_s**(-e_s)*(j_b + 1)**(-e_b)*(j_s + 1)**e_s/a_b

In [25]:
# Partial derivatives
dL_des = L.diff(e_s)
dL_deb = L.diff(e_b)
dL_dl1 = L.diff(l1)
dL_dl2 = L.diff(l2)
# dL_dl3 = L.diff(l3)
# dL_dl4 = L.diff(l4)
# dL_dl5 = L.diff(l5)
# dL_dl6 = L.diff(l6)

# Set them to zero
stationary_conditions = [
    Eq(dL_des, 0),
    Eq(dL_deb, 0),
    Eq(dL_dl1, 0),
    Eq(dL_dl2, 0),
    # Eq(dL_dl3, 0),
    # Eq(dL_dl4, 0),
    # Eq(dL_dl5, 0),
    # Eq(dL_dl6, 0),
]

In [26]:
solutions_ = solve(stationary_conditions, (e_s, e_b, l1, l2))#, l3, l4, l5, l6))

print(len(solutions_))
solutions = solutions_[0]
# e_s_sol, e_b_sol, l1_sol, l2_sol, l3_sol, l4_sol, l5_sol, l6_sol = solutions
e_s_sol, e_b_sol, l1_sol, l2_sol = solutions
solutions

1


(log((w_s*(b_s + v_s)/a_s)**(1/log((j_s + 1)/j_s))),
 log((w_b*(b_b + v_b)/a_b)**(1/log((j_b + 1)/j_b))),
 log(((j_b + 1)/j_b)**(a_s*j_b**log((w_b*(b_b + v_b)/a_b)**(1/log((j_b + 1)/j_b)))*j_s**log((a_s/(w_s*(b_s + v_s)))**(1/log((j_s + 1)/j_s)))*(j_b + 1)**log((a_b/(w_b*(b_b + v_b)))**(1/log((j_b + 1)/j_b)))*(j_s + 1)**log((w_s*(b_s + v_s)/a_s)**(1/log((j_s + 1)/j_s)))/a_b)),
 log(((j_s + 1)/j_s)**(a_s*j_b**log((w_b*(b_b + v_b)/a_b)**(1/log((j_b + 1)/j_b)))*j_s**log((a_s/(w_s*(b_s + v_s)))**(1/log((j_s + 1)/j_s)))*(j_b + 1)**log((a_b/(w_b*(b_b + v_b)))**(1/log((j_b + 1)/j_b)))*(j_s + 1)**log((w_s*(b_s + v_s)/a_s)**(1/log((j_s + 1)/j_s)))/a_b)))

In [27]:
e_s_sol;

log((w_s*(b_s + v_s)/a_s)**(1/log((j_s + 1)/j_s)))

In [28]:
e_b_sol;

log((w_b*(b_b + v_b)/a_b)**(1/log((j_b + 1)/j_b)))

In [29]:
l1_sol;

log(((j_b + 1)/j_b)**(a_s*j_b**log((w_b*(b_b + v_b)/a_b)**(1/log((j_b + 1)/j_b)))*j_s**log((a_s/(w_s*(b_s + v_s)))**(1/log((j_s + 1)/j_s)))*(j_b + 1)**log((a_b/(w_b*(b_b + v_b)))**(1/log((j_b + 1)/j_b)))*(j_s + 1)**log((w_s*(b_s + v_s)/a_s)**(1/log((j_s + 1)/j_s)))/a_b))

In [30]:
l2_sol;

log(((j_s + 1)/j_s)**(a_s*j_b**log((w_b*(b_b + v_b)/a_b)**(1/log((j_b + 1)/j_b)))*j_s**log((a_s/(w_s*(b_s + v_s)))**(1/log((j_s + 1)/j_s)))*(j_b + 1)**log((a_b/(w_b*(b_b + v_b)))**(1/log((j_b + 1)/j_b)))*(j_s + 1)**log((w_s*(b_s + v_s)/a_s)**(1/log((j_s + 1)/j_s)))/a_b))

In [31]:
for sol in solutions:
  print(sol.free_symbols)

{v_s, w_s, b_s, j_s, a_s}
{v_b, a_b, b_b, w_b, j_b}
{b_s, b_b, a_s, j_b, v_b, v_s, w_s, a_b, j_s, w_b}
{b_s, b_b, a_s, j_b, v_b, v_s, w_s, a_b, j_s, w_b}


In [37]:
u1, u2, u3, u4, u5, u6, u7 = symbols('u1 u2 u3 u4 u5 u6 u7')

exp = (a_s*j_b**log(u3**u1)*j_s**log(u5**u2)*(j_b + 1)**log(u6**u1)*(j_s + 1)**log(u4**u2)/a_b)
subs1 = l1_sol.subs(1/log((j_b + 1)/j_b), u1)
subs1 = subs1.subs(1/log((j_s + 1)/j_s), u2)
subs1 = subs1.subs(w_b*(b_b + v_b)/a_b, u3)
subs1 = subs1.subs(w_s*(b_s + v_s)/a_s, u4)
subs1 = subs1.subs(a_s/(w_s*(b_s + v_s)), u5)
subs1 = subs1.subs(a_b/(w_b*(b_b + v_b)), u6)
subs1 = subs1.subs(exp, u7)

subs1 # !>= 0

log(((j_b + 1)/j_b)**u7)

In [38]:
subs2 = l2_sol.subs(1/log((j_b + 1)/j_b), u1)
subs2 = subs2.subs(1/log((j_s + 1)/j_s), u2)
subs2 = subs2.subs(w_b*(b_b + v_b)/a_b, u3)
subs2 = subs2.subs(w_s*(b_s + v_s)/a_s, u4)
subs2 = subs2.subs(a_s/(w_s*(b_s + v_s)), u5)
subs2 = subs2.subs(a_b/(w_b*(b_b + v_b)), u6)
subs2 = subs2.subs(exp, u7)

subs2 # !>= 0

log(((j_s + 1)/j_s)**u7)

In [39]:
exp # >0 --> log-arg > 1 --> log > 0 --> both lambdas > 0 qed

a_s*j_b**log(u3**u1)*j_s**log(u5**u2)*(j_b + 1)**log(u6**u1)*(j_s + 1)**log(u4**u2)/a_b

### complementary slackness

given that the lambdas are strictly positive, we need to show that the
associated constraints are zero at the solution

In [59]:
simplify(expBound["b"].subs({ e_b: e_b_sol }));

0

In [60]:
simplify(expBound["s"].subs({ e_s: e_s_sol }));

0

qed

## Consequences

In [46]:
Delta_b_sol = deltaByExp["b"].subs(e_s, e_s_sol).subs(e_b, e_b_sol);
Delta_b_sol;

a_b*j_b**(-log((w_b*(b_b + v_b)/a_b)**(1/log((j_b + 1)/j_b))))*(j_b + 1)**log((w_b*(b_b + v_b)/a_b)**(1/log((j_b + 1)/j_b)))/w_b - b_b - v_b

In [48]:
Delta_b_sol2 = a_b *
    ((j_b + 1) / j_b) **
      log((w_b * (b_b + v_b) / a_b) ** (1 / log((j_b + 1) / j_b))) /
    w_b - b_b - v_b;
Delta_b_sol2;

a_b*((j_b + 1)/j_b)**log((w_b*(b_b + v_b)/a_b)**(1/log((j_b + 1)/j_b)))/w_b - b_b - v_b

In [49]:
simplify(Delta_b_sol - Delta_b_sol2);

0

In [51]:
Delta_s_sol = deltaByExp["s"].subs(e_s, e_s_sol).subs(e_b, e_b_sol);
Delta_s_sol;

a_s*j_s**(-log((w_s*(b_s + v_s)/a_s)**(1/log((j_s + 1)/j_s))))*(j_s + 1)**log((w_s*(b_s + v_s)/a_s)**(1/log((j_s + 1)/j_s)))/w_s - b_s - v_s

In [55]:
# same as Delta_b_sol2 with indices swapped (literally how it was typed out)
Delta_s_sol2 = a_s*((j_s + 1)/j_s)**log((w_s*(b_s + v_s)/a_s)**(1/log((j_s + 1)/j_s)))/w_s - b_s - v_s
Delta_s_sol2

a_s*((j_s + 1)/j_s)**log((w_s*(b_s + v_s)/a_s)**(1/log((j_s + 1)/j_s)))/w_s - b_s - v_s

In [56]:
simplify(Delta_s_sol - Delta_s_sol2);

0